# 神经网络工具箱Torch.nn第一节：构造模型的方法
torch.nn其是专门为深度学习而设计的模块。torch.nn的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个**包含很多层的神经网络**。在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。

----
**本节主要参考资料为：**apacheCN的[pytorch文档](https://pytorch.apachecn.org/docs/1.2/)、ShusenTang改写的pytorch版[《动手学深度学习》](http://tangshusen.me/Dive-into-DL-PyTorch/#/ '《动手学深度学习》')和陈云的[《深度学习框架PYTORCH入门与实践》](https://github.com/chenyuntc/pytorch-book)

In [1]:
import torch as t
from torch import nn #import torch.nn as nn
import torch.nn.functional as F

**一个神经网络的典型训练过程如下：**

- 定义包含一些可学习参数（或者叫权重）的神经网络
- 在输入数据集上迭代
- 通过网络处理输入
- 计算损失（输出和正确答案的距离）
- 将梯度反向传播给网络的参数
- 更新网络的权重，一般使用一个简单的规则：`weight = weight - learning_rate * gradient`

## 继承`Module`类来构造模型
下面继承`Module`类构造多层感知机，定义的`MLP`类重载了`Module`类的`__init__`函数和`forward`函数，分别用于创建模型参数和定义前向计算(正向传播)

In [11]:
class MLP(nn.Module):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        # super()调用MLP父类Module的构造函数__init__进行初始化，
        # 这样在构造实例时还可以指定其他函数的的参数
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU() #激活函数
        self.output =nn.Linear(256,10) # 输出层
        
    def forward(self,x):
        a=self.act(self.hidden(x))

以上的`MLP`类中无须定义反向传播函数。系统将通过**自动求梯度**而自动生成反向传播所需的`backward`函数。

----
>\*args和\**kwargs主要用于函数定义。你可以将不定数量的参数传递给某个函数

>\*args是用来发送一个非键值对的可变数量的参数列表给一个函数.

>\**kwargs允许你将不定长度的键值对作为参数传递给一个函数。如果你想要在一个函数里处理带名字的参数，你应该使用\**kwargs

----
我们可以实例化`MLP`类得到**模型变量**`net`。下面的代码初始化`net`并传入输入数据X做一次前向计算。<font color=red>其中，net(X)会调用MLP继承自Module类的``__call__``函数，这个函数将调用MLP类定义的`forward`函数来完成前向计算。</font>



In [16]:
net=MLP()
X=t.rand(2,784)
net(X)
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


注意，这里并没有将`Module`类命名为`Layer`（层）或者`Model`（模型）之类的名字，这是因为该类是一个**可供自由组建**的部件。它的子类既可以是一个层（如PyTorch提供的Linear类），又可以是一个模型（如这里定义的MLP类），或者是模型的一个部分
## `Module`的子类
Module类是一个通用的部件，PyTorch还实现了继承自Module的可以方便构建模型的类: 如Sequential、ModuleList和ModuleDict等等
### `Sequential`类
`Sequential`类可以接收一个子模块的**有序字典（OrderedDict）或者一系列子模块作为参数**来逐一添加`Module`的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

下面我们实现一个与`Sequential`类有相同功能的`MySequential`类：

In [20]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args) == 1 and isinstance(args[0],OrderedDict):# 如果传入OrderedDict
            for key, module in args[0].items():
                self.add_module(key,module) # add_module方法会将module添加进self._modules(一个OrderedDict)
        else: #传入的不是OrderedDict而是Module
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
    def forward(self,input):
        for module in self._modules.values():
            input=module(input)
        return input
net = MySequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1460,  0.1617, -0.1478,  0.2251, -0.1415,  0.0834,  0.0942,  0.2068,
          0.1312, -0.1836],
        [ 0.0809,  0.1665, -0.1793,  0.1542, -0.0970,  0.1232,  0.1412,  0.2647,
          0.2444, -0.0876]], grad_fn=<AddmmBackward>)

`isinstance() `函数来判断一个对象是否是一个已知的类型，`isinstance(object, classinfo)`

isinstance() 与 type() 区别：
- type() 不会认为子类是一种父类类型，不考虑继承关系。
- isinstance() 会认为子类是一种父类类型，考虑继承关系。

`enumerate() `函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出**数据和数据下标idx**，一般用在 for 循环当中

`Linear`即对输入的数据进行线性变换：`class torch.nn.Linear(in_features, out_features, bias=True)`

Applies a linear transformation to the incoming data:$y=x{A}^{T}+b$

### `ModuleList`类
`ModuleList`接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作:

In [21]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # # 类似List的append操作
print(net[-1])  # 类似List的索引访问
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


既然Sequential和ModuleList都可以进行列表化构造网络，那二者区别是什么呢。

ModuleList仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序（所以不用保证相邻层的输入输出维度匹配），而且没有实现forward功能需要自己实现，所以上面执行net(torch.zeros(1, 784))会报NotImplementedError；而Sequential内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配，内部forward功能已经实现。

`ModuleList`的出现只是让网络定义前向传播时更加灵活，见下面官网的例子

In [22]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

另外，ModuleList不同于一般的Python的list，加入到ModuleList里面的所有模块的参数会被自动添加到整个网络中，下面看一个例子对比一下。

In [23]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p)

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


### `ModuleDict`类
`ModuleDict`接收一个子模块的字典作为输入, 然后也可以**类似字典那样**进行添加访问操作:

In [24]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 构造复杂的模型
虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义`forward`函数，但**直接继承Module类可以极大地拓展模型构造的灵活性**。下面我们构造一个稍微复杂点的网络FancyMLP。在这个网络中，我们通过`get_constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用Tensor的函数和Python的控制流，并多次调用相同的层

In [ ]:
class FancyMLP(nn.Module):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        
        self.rand_weight=t.rand((20,20),requires_grad=False) #不可训练参数，自然grad也设置为False
        slef.linear=nn.Linear(20,20)
        
    def forward(self,x):
        x=self.linear(x)# 经过一次线性运算